# Joining and Validating ExIm Data

### Joining the data
Concatenate all of the export/import files into one dataframe with a common set of columns, optionally writing to csv as `data.csv`.

### Cleaning the data
 - Need to homogenize the HS Descriptions so that there's a 1-1 mapping between them and the HS Codes.
 - EDA to find other issues

In [1]:
import pandas as pd
import os
import numpy as np
from itertools import combinations
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import re

In [2]:
def parse_and_concat_files(write=False):
    """
    Get a list of the excel files within the `Export and Import Data` dir,
    convert them to dataframes, add some columns indicating import/export,
    the HS2 Code, and the HS2 Description from chapter.xls. Optionally
    write the data to your current working directory.

    Returns:
        data (pandas DataFrame):
    """

    col_types = {'CIF Value (ETB)':float,
             'CIF Value (USD)':float,
             'CPC':str,
             'Country (Consignment)':str,
             'Country (Origin)':str,
             'Destination':str,
             'FOB Value (ETB)':float,
             'FOB Value (USD)':float,
             'Gross Wt. (Kg)':float,
             'HS Code':str,
             'HS Description':str,
             'Month':str,
             'Net Wt. (Kg)':float,
             'Net.Wt. (Kg)':float,
             'Quantity':float,
             'Sup. Unit':str,
             'Total tax (ETB)':float,
             'Total tax (USD)':float,
             'Unit':str,
             'Year':str}

    files = []
    for dirpath, dirnames, filenames in os.walk(os.getcwd()):
        for file in filenames:
            if '.xls' in file and 'chapter' not in file and "~$" not in file:
                files.append(file)

    chapter = pd.read_excel(r'chapter.xls',dtype=str)
    dfs = []
    files_len = len(files)
    for i, file in enumerate(files):
        df = pd.read_excel(file,dtype=col_types)
        if 'import' in file:
            df['Destination'] = np.nan
        else:
            df['Country (Origin)'] = np.nan
            df['Country (Consignment)'] = np.nan

        df.rename({'Net.Wt. (Kg)':'Net Wt. (Kg)',
                   'CIF Value (USD)':'Value (USD)',
                   'CIF Value (ETB)':'Value (ETB)',
                   'FOB Value (ETB)':'Value (ETB)',
                   'FOB Value (USD)':'Value (USD)',
                   'Sup. Unit':'Unit'},axis=1,inplace=True)
        df['Direction'] = file[:file.index("_")]
        df['HS2'] = df['HS Code'].apply(lambda x: x[:2])
        df = pd.merge(df,chapter[['HS2','HS2_DSCRIPTION']],on='HS2')
        if i > 0:
            last_cols = list(dfs[i-1].columns)
            current_cols = sorted(df.columns)
            assert (last_cols == current_cols),"Column names weren't the same!"

        dfs.append(df[sorted(df.columns)])
        print("Done parsing {} of {} files".format(i+1,files_len))

    data = pd.concat(dfs)
    if write:
        data.to_csv('data.csv',index=False)

    return data

In [3]:
data = parse_and_concat_files()

Done parsing 1 of 11 files
Done parsing 2 of 11 files
Done parsing 3 of 11 files
Done parsing 4 of 11 files
Done parsing 5 of 11 files
Done parsing 6 of 11 files
Done parsing 7 of 11 files
Done parsing 8 of 11 files
Done parsing 9 of 11 files
Done parsing 10 of 11 files
Done parsing 11 of 11 files


## Data Validation
In this section, we'll first normalize the HS Descriptions by stripping non-alphas and whitespace before converting to lower case. We'll call these the `Normalized HS Description`. This will facilitate string comparisons.

Then we'll identify the HS Codes that map to more than one `Normalized HS Description`. For each of these HS Codes, we'll look at all of the 2-way combinations of corresponding HS Descriptiosn and calcuate a partial-ratio string similarity score. These will map back to each HS Code in a dictionary  called `codes_descriptions_map`. The keys will be the HS Codes, and each value will be a list containing a dict that maps each 2-way combination to its partial-ratio score.

We can visually inspect `codes_descriptions_map` and then determine what to do.

In [4]:
def normalize_text(text):
    """
    Strip non-alpha chars from string, strip whitespace, and lowercase it.
    """
    alphas = re.compile(r'[a-zA-Z]*')
    nearly_normal_string = " ".join((s.lower() for s in alphas.findall(text))).strip(' \t\n\r')
    return ' '.join(nearly_normal_string.split())

In [5]:
data['Normalized HS Description'] = data['HS Description'].apply(normalize_text)

In [6]:
#get HS Codes that map to more than one Normalized HS Description
one_to_many = data.groupby(by = 'HS Code')['Normalized HS Description'].nunique()[data.groupby(by = 'HS Code')['Normalized HS Description'].nunique() > 1]



For each HS Description combination, compute the patial_ratio string similarity score. Use this instead of ratio since inconsistent substrings are common. The “best partial” heuristic will take two strings - the shorter string  of length $m$ and the longer string of length $n$ - and score the best matching length-$m$ substring.

In [7]:
hs_codes = one_to_many.index

codes_descriptions_map = {k:[] for k in hs_codes}
for hs_code in hs_codes:
    hs_descriptions = list(data[data['HS Code']==hs_code]['Normalized HS Description'].unique())
    hs_description_combos = list(combinations(hs_descriptions, 2))
    for combo in hs_description_combos:
        ratio = fuzz.partial_ratio(combo[0], combo[1])
        descriptions = "-".join(combo)
        codes_descriptions_map[hs_code].append({descriptions:ratio})

In [176]:
codes_descriptions_map

{'03055900': [{'dried fish not smoked excl cod-other': 40}],
 '03056900': [{'other fish salted or in brine but not dried or smoked nes-other': 100}],
 '20029090': [{'other tomato concentrate-other': 100}],
 '20049000': [{'vegetables nes preserved not by vinegar etc frozen excl prods of-other vegetables and mixtures of vegetables': 56}],
 '20055900': [{'beans unshelled preserved other than by vinegar etc not frozen-other': 100}],
 '20059900': [{'other-other vegetables and mixtures': 100}],
 '20071000': [{'jams fruit jellies marmalades etc homogenized-homogenised preparations': 57}],
 '20081100': [{'ground nuts preserved-ground nuts': 100}],
 '20081900': [{'other including mixtures-nuts and seeds including mixtures preserved': 83}],
 '20087000': [{'peaches prepared or preserved excl those of and-peaches including nectarines': 46}],
 '20091910': [{'orange concentrate-orange concentrate other': 100}],
 '20091990': [{'other orange concentrate-other': 100}],
 '20092100': [{'grapefruit juice 

After scanning the above, it seems like the best way to standardize the HS Code-HS Descrption mapping is to use the longest Normalized HS Description for each HS Code.

In [8]:
codes = data['HS Code'].unique()
code_description_map = {k:None for k in codes}
for code in code_description_map:
    longest_desc = max(list(data[data['HS Code'] == code]['Normalized HS Description'].unique()),key=len)
    code_description_map[code] = longest_desc

In [9]:
data['Standardized HS Description'] = data['HS Code'].map(code_description_map)

In [10]:
data.to_csv('data.csv',index=False)

## Analysis

In [23]:
data.head()

,CPC,Country (Consignment),Country (Origin),Destination,Direction,Gross Wt. (Kg),HS Code,HS Description,HS2,HS2_DSCRIPTION,...,Net Wt. (Kg),Quantity,Total tax (ETB),Total tax (USD),Unit,Value (ETB),Value (USD),Year,Normalized HS Description,Standardized HS Description
0,1000 100,NaN,NaN,Switzerland,export,1207.60,71081300,Semi-manufactured gold (incl. gold plated with...,71,"NATURAL/CULTURED PEARLS, PREC STONES & METALS,...",...,1092.85,NaN,0.0,0.0,nan,8.145915e+08,4.332910e+07,2013,semi manufactured gold incl gold plated with p...,other semi manufacturedforms gold incl gold pl...
1,1000 100,NaN,NaN,India,export,1183.00,71031000,Precious (excl. diamonds) or semi-precious sto...,71,"NATURAL/CULTURED PEARLS, PREC STONES & METALS,...",...,1095.44,NaN,0.0,0.0,nan,4.579759e+06,2.436029e+05,2013,precious excl diamonds or semi precious stones...,precious excl diamonds or semi precious stones...
2,1000 100,NaN,NaN,United States,export,228.78,71031000,Precious (excl. diamonds) or semi-precious sto...,71,"NATURAL/CULTURED PEARLS, PREC STONES & METALS,...",...,211.35,NaN,0.0,0.0,nan,4.261545e+06,2.266767e+05,2013,precious excl diamonds or semi precious stones...,precious excl diamonds or semi precious stones...
3,1000 100,NaN,NaN,Thailand,export,45.00,71031000,Precious (excl. diamonds) or semi-precious sto...,71,"NATURAL/CULTURED PEARLS, PREC STONES & METALS,...",...,40.58,NaN,0.0,0.0,nan,3.659941e+05,1.946767e+04,2013,precious excl diamonds or semi precious stones...,precious excl diamonds or semi precious stones...
4,1000 100,NaN,NaN,Singapore,export,28.20,71031000,Precious (excl. diamonds) or semi-precious sto...,71,"NATURAL/CULTURED PEARLS, PREC STONES & METALS,...",...,25.83,NaN,0.0,0.0,nan,2.910946e+05,1.548367e+04,2013,precious excl diamonds or semi precious stones...,precious excl diamonds or semi precious stones...


In [14]:
imports = data[data['Direction']=='import']
exports = data[data['Direction']=='export']

In [19]:
imports.groupby(by=['Country (Consignment)','Year'])['Value (USD)'].sum().sort_values(ascending=False)

Country (Consignment)     Year
China                     2015    5.342251e+09
                          2014    4.604887e+09
                          2016    4.345332e+09
                          2017    4.196616e+09
                          2013    2.530710e+09
Djibouti                  2017    1.282495e+09
Saudi Arabia              2014    1.206669e+09
Djibouti                  2016    1.117733e+09
                          2015    1.093989e+09
United States             2016    1.058246e+09
United Arab Emirates      2016    1.034168e+09
India                     2013    1.023856e+09
United Arab Emirates      2015    1.001527e+09
                          2014    9.996963e+08
India                     2016    9.910588e+08
Kuwait                    2016    9.673827e+08
United Arab Emirates      2013    9.623376e+08
Kuwait                    2014    9.425649e+08
                          2017    8.909492e+08
India                     2015    8.546774e+08
United Arab Emirates      201

In [26]:
exports.head()

,CPC,Country (Consignment),Country (Origin),Destination,Direction,Gross Wt. (Kg),HS Code,HS Description,HS2,HS2_DSCRIPTION,...,Net Wt. (Kg),Quantity,Total tax (ETB),Total tax (USD),Unit,Value (ETB),Value (USD),Year,Normalized HS Description,Standardized HS Description
0,1000 100,NaN,NaN,Switzerland,export,1207.60,71081300,Semi-manufactured gold (incl. gold plated with...,71,"NATURAL/CULTURED PEARLS, PREC STONES & METALS,...",...,1092.85,NaN,0.0,0.0,nan,8.145915e+08,4.332910e+07,2013,semi manufactured gold incl gold plated with p...,other semi manufacturedforms gold incl gold pl...
1,1000 100,NaN,NaN,India,export,1183.00,71031000,Precious (excl. diamonds) or semi-precious sto...,71,"NATURAL/CULTURED PEARLS, PREC STONES & METALS,...",...,1095.44,NaN,0.0,0.0,nan,4.579759e+06,2.436029e+05,2013,precious excl diamonds or semi precious stones...,precious excl diamonds or semi precious stones...
2,1000 100,NaN,NaN,United States,export,228.78,71031000,Precious (excl. diamonds) or semi-precious sto...,71,"NATURAL/CULTURED PEARLS, PREC STONES & METALS,...",...,211.35,NaN,0.0,0.0,nan,4.261545e+06,2.266767e+05,2013,precious excl diamonds or semi precious stones...,precious excl diamonds or semi precious stones...
3,1000 100,NaN,NaN,Thailand,export,45.00,71031000,Precious (excl. diamonds) or semi-precious sto...,71,"NATURAL/CULTURED PEARLS, PREC STONES & METALS,...",...,40.58,NaN,0.0,0.0,nan,3.659941e+05,1.946767e+04,2013,precious excl diamonds or semi precious stones...,precious excl diamonds or semi precious stones...
4,1000 100,NaN,NaN,Singapore,export,28.20,71031000,Precious (excl. diamonds) or semi-precious sto...,71,"NATURAL/CULTURED PEARLS, PREC STONES & METALS,...",...,25.83,NaN,0.0,0.0,nan,2.910946e+05,1.548367e+04,2013,precious excl diamonds or semi precious stones...,precious excl diamonds or semi precious stones...


In [35]:
exports['Country (Consignment)'].notnull().sum()

0

1231146